In [5]:
import response
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_features = 6000
max_len =  130
embedding_size = 128

(input_train, label_train), (input_test, label_test) = imdb.load_data(num_words=max_features)
input_train = pad_sequences(input_train, maxlen=max_len)
input_test = pad_sequences(input_test, maxlen=max_len)

'''
지시사항 1번
텐서 데이터에 대해 Convolution 연산을 진행하고 출력하는 함수를 만들어 봅니다.(좌측 설명의 그림을 참고)
'''
    
def conv(_input, node=16):
#     _input = tf.cast(_input, tf.float32)
    _input = expand_dim(_input)
    conv1= tf.keras.layers.Conv1D( filters=node, kernel_size=1, strides=1, padding='same')
    conv2= tf.keras.layers.Conv1D( filters=node, kernel_size=2, strides=1, padding='same')
    conv3= tf.keras.layers.Conv1D( filters=node, kernel_size=3, strides=1, padding='same')
    conv1 = conv1(_input)
    conv2 = conv2(_input)
    conv3 = conv3(_input)
    concat = tf.keras.layers.concatenate([conv1, conv2, conv3], axis=1)
    return concat


def expand_dim(_input):
    return tf.expand_dims(_input, -1)
    
'''
지시사항 2~3번
2. LSTM 기반의 Bidirectional층을 추가합니다.(return_sequences = True)
3. Lambda 함수를 활용하여 conv 함수를 레이어로 추가시켜 줍니다.
'''
def main():
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(max_features, embedding_size),
        tf.keras.layers.Bidirectional(layer=tf.keras.layers.LSTM(units=5, return_sequences=True)),
        tf.keras.layers.GlobalMaxPool1D(),
        tf.keras.layers.Lambda(conv),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation = tf.nn.sigmoid, dtype=tf.float32),
    ])

    model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['acc'])
    
    model.summary()
    
    history = model.fit(input_train, label_train, validation_split=.2, batch_size=100, epochs=5,verbose=2)

if __name__=='__main__':
    main()

ValueError: 
The following Variables were created within a Lambda layer (lambda_4)
but are not tracked by said layer:
  <tf.Variable 'lambda_4/conv1d/kernel:0' shape=(1, 1, 16) dtype=float32>
  <tf.Variable 'lambda_4/conv1d/bias:0' shape=(16,) dtype=float32>
  <tf.Variable 'lambda_4/conv1d_1/kernel:0' shape=(2, 1, 16) dtype=float32>
  <tf.Variable 'lambda_4/conv1d_1/bias:0' shape=(16,) dtype=float32>
  <tf.Variable 'lambda_4/conv1d_2/kernel:0' shape=(3, 1, 16) dtype=float32>
  <tf.Variable 'lambda_4/conv1d_2/bias:0' shape=(16,) dtype=float32>
The layer cannot safely ensure proper Variable reuse across multiple
calls, and consquently this behavior is disallowed for safety. Lambda
layers are not well suited to stateful computation; instead, writing a
subclassed Layer is the recommend way to define layers with
Variables.